In [23]:
from IPython.display import SVG, display, HTML
from tqdm import tqdm
from pprint import pprint
from argparse import Namespace
import pandas as pd
import ujson
import igraph as ig
import enum
from pathlib import Path

from pywikidata import Entity


import time
import sys
import os
import psutil
import networkx as nx
import gc


In [24]:
class SubgraphNodeType(str, enum.Enum):
    """SubgraphNodeType - Enum class with types of subgraphs nodes"""

    INTERNAL = "Internal"
    QUESTIONS_ENTITY = "Question entity"
    ANSWER_CANDIDATE_ENTITY = "Answer candidate entity"

In [25]:
# Labeling vertexes and edges in mined subgraphs
split = 'test'
dataset = 'mintaka'
with open(f'./{dataset}_results_procs/{dataset}_{split}_labeled.jsonl', 'w') as f_out:
    with open(f'./{dataset}_results_procs/{dataset}_{split}.jsonl', 'r') as f:
        for line in tqdm(f):
            data = ujson.loads(line)

            for node in data['graph']['nodes'] + data['graph']['links']:
                node['label'] = Entity(node['name_']).label

            f_out.write(ujson.dumps(data)+'\n')

28325it [00:01, 14195.43it/s]


In [26]:
df = []
with open(f'./{dataset}_results_procs/{dataset}_{split}_labeled.jsonl', 'r') as f:
    for idx, line in enumerate(f):
        data = ujson.loads(line)
        df.append(data)

df = pd.DataFrame(df)
df.head()

,id,question,answerEntity,questionEntity,groundTruthAnswerEntity,complexityType,graph
0,fae46b21,What man was a famous American author and also...,[Q893594],"[Q1497, Q846570]",[Q7245],intersection,"{'directed': True, 'multigraph': False, 'graph..."
1,fae46b21,What man was a famous American author and also...,[Q102513],"[Q1497, Q846570]",[Q7245],intersection,"{'directed': True, 'multigraph': False, 'graph..."
2,fae46b21,What man was a famous American author and also...,[Q7245],"[Q1497, Q846570]",[Q7245],intersection,"{'directed': True, 'multigraph': False, 'graph..."
3,fae46b21,What man was a famous American author and also...,[Q34652890],"[Q1497, Q846570]",[Q7245],intersection,"{'directed': True, 'multigraph': False, 'graph..."
4,fae46b21,What man was a famous American author and also...,[Q5686],"[Q1497, Q846570]",[Q7245],intersection,"{'directed': True, 'multigraph': False, 'graph..."


In [28]:
df['complexityType'].unique()

array(['intersection', 'count', 'comparative', 'ordinal', 'multihop',
       'difference', 'superlative', 'generic'], dtype=object)

In [29]:
if dataset == 'mintaka':
    df = df[df['complexityType'] != 'count']

In [30]:
groups = df.groupby('question')

In [31]:
def nx_subgraph_to_svg_plt(nx_subgraph, correct_answer_entities: list[Entity]):
    for node in nx_subgraph.nodes:
        if nx_subgraph.nodes[node]['type'] == str(SubgraphNodeType.ANSWER_CANDIDATE_ENTITY).split('.')[-1]:
            nx_subgraph.nodes[node]['style'] = 'filled'
            if nx_subgraph.nodes[node]['name_'] in [correct_answer_entity.idx for correct_answer_entity in correct_answer_entities]:
                nx_subgraph.nodes[node]['fillcolor'] = 'green'
            else:
                nx_subgraph.nodes[node]['fillcolor'] = 'coral'
        elif nx_subgraph.nodes[node]['type'] == str(SubgraphNodeType.QUESTIONS_ENTITY).split('.')[-1]:
            nx_subgraph.nodes[node]['style'] = 'filled'
            nx_subgraph.nodes[node]['fillcolor'] = 'deepskyblue'
        else:
            nx_subgraph.nodes[node]['style'] = 'filled'
            nx_subgraph.nodes[node]['fillcolor'] = 'lightgrey'

    agraph = nx.nx_agraph.to_agraph(nx_subgraph)
    plt = agraph.draw(prog='dot',format='svg')
    return plt

In [32]:
def viz_question(q):
    group = groups.get_group(q)

    plts = []
    correct_answer_entity = [Entity(e) for e in group.iloc[0]['groundTruthAnswerEntity']]
    for graph in group.graph:
        nx_subgraph = nx.readwrite.json_graph.node_link_graph(graph)
        plts.append(nx_subgraph_to_svg_plt(nx_subgraph, correct_answer_entity))

    subgraphs_svg = [SVG(p).data for p in plts]

    html = """
    <style>
    .flex-row-container {
        # background: #aaa;
        display: flex;
        flex-wrap: wrap;
        align-items: center;
        justify-content: center;
    }
    .flex-row-container > .flex-row-item {
        flex: 1 1 30%; /*grow | shrink | basis */
        height: 500px;
    }

    .flex-row-item {
    # background-color: #fff4e6;
    border: 1px solid #f76707;
    }
    </style>
    """
    html += f"<h1>{q}</h1>"
    html += f"<br>Correct answer: <b>"
    html += "; ".join([str(e.label) + f'({e.idx})' for e in correct_answer_entity])
    html += '<div class="flex-row-container">'
    for svg in subgraphs_svg:
        html += '<div class="flex-row-item">{}</div>'.format(svg)
    html += '</div>'

    if dataset == 'mintaka':
        return html, group['id'].iloc[0]
    else:
        return html, None

In [33]:
uni_questions = df.question.unique()
for i in range(30):
    print(f'{i:2d}) ', uni_questions[i])

 0)  What man was a famous American author and also a steamboat pilot on the Mississippi River?
 1)  Who is older, The Weeknd or Drake?
 2)  Which movie, starring Al Jolson, is generally considered to be the first talking picture?
 3)  Which movie had a bigger budget, Avatar or Training Day?
 4)  What state is the author of Misery from?
 5)  Which movie sold more tickets: Titanic or Avatar?
 6)  Who is older, Martin Scorsese or David Fincher?
 7)  Which movie came out first, Monsters, Inc. or Lilo and Stitch?
 8)  Who is taller, George Clooney or Brad Pitt?
 9)  Who is older, Chris Pratt or Chris Evans?
10)  Who is older, Chris Evans or Anna Faris?
11)  Which movie had a higher box office return, Moon or Eternal Sunshine of a Spotless Mind?
12)  Who is younger, Emma Watson or Saoirse Ronan?
13)  Which movie came out first, Transformers or Toy Story 2?
14)  Which movie had a higher domestic gross, Iron Man or Iron Man 2?
15)  Who is younger, Drew Barrymore or Reese Witherspoon?
16)  Whi

In [35]:
html, _ = viz_question(uni_questions[12])
display(HTML(html))

In [20]:
import imgkit
(Path(f'./{dataset}_results_procs/{dataset}_{split}_labeled.jsonl').parent / 'viz' / split).mkdir(parents=True, exist_ok=True)

In [22]:
for question in tqdm(uni_questions):
    try:
        html, idx = viz_question(question)
        img_path = Path(f'./{dataset}_results_procs/{dataset}_{split}_labeled.jsonl').parent / 'viz' / split / f"{idx}_{question.replace(' ', '_').replace('?', '')}.png"
        imgkit.from_string(html, img_path, options={"xvfb": "", "format": "png"})
    except Exception as e:
        print(e)

  0%|          | 1/10887 [00:00<2:02:16,  1.48it/s]

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


  0%|          | 2/10887 [00:01<1:49:15,  1.66it/s]

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


  0%|          | 3/10887 [00:01<1:52:46,  1.61it/s]

...

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


  3%|▎         | 336/10887 [03:40<1:52:09,  1.57it/s]

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


  3%|▎         | 336/10887 [03:41<1:56:01,  1.52it/s]


KeyboardInterrupt: 